# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from ml_core.normalization import z_score_normalization
from ml_core.data_process import *
from ml_core.regression import GradientDescent, OrdinaryLeastSquares, StochasticGradientDescent
from sklearn.model_selection import train_test_split
from ml_core.metrics.error_metrics import *
from ml_core.metrics import error_metric_in_history_models
from ml_core.metrics.error_metrics import *
from ml_core.classification import LogisticBinaryModel,LinearRegressionMulticlass
from ml_core.classification import StatisticalClassifiers, GaussianDiscriminant, NaiveBayesGaussian
from sklearn.naive_bayes import GaussianNB

# Leitura dos Dados

In [2]:
df = pd.read_csv('data/breastcancer.csv', header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0.0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0.0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0.0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0.0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0.0


# Preparar os dados para experimentos


In [4]:
arr = df.to_numpy()

In [5]:
X = arr[:, :30]
Y = arr[:, -1]

In [6]:
X_norm = np.empty_like(X)

for idx, col in enumerate(X.T):
    
    norm = z_score_normalization(col)
    
    X_norm[:, idx] = norm

In [7]:
y_norm = Y.reshape([-1, 1])

In [8]:
X_ones = add_ones_column(X_norm)

In [9]:
X_trn, X_tst, y_trn, y_tst = train_test_split(X_ones, y_norm, test_size=0.2, random_state=42)

# Classificação Binária

## Regressão Logistica

In [10]:
first_model = LogisticBinaryModel.first_model(
    lenght=X_trn.shape[1],
    fill_value=1
)

In [11]:
gd = GradientDescent(ephocs=1000)

In [12]:
gd_model = gd.fit(X_trn, y_trn, first_model=first_model.__copy__())

  0%|          | 0/1000 [00:00<?, ?it/s]

In [13]:
preds = gd_model.predict_ajust(X_tst)

In [14]:
acuracy(y_tst, preds), f1_score(y_tst, preds), precision(y_tst, preds)

(0.956140350877193, 0.7777777777777777, 0.6422018348623854)

## Analise Discriminante Gaussiano

In [15]:
y_trn.T.dtype

dtype('float64')

In [16]:
X_trn_ = X_trn[:, 1:]  
y_trn_dummies = get_dummies(y_trn.reshape([-1]))

In [17]:
gaussian_discriminant = StatisticalClassifiers(model=GaussianDiscriminant)

In [18]:
model_gaussian = gaussian_discriminant.fit(X_trn_, y_trn_dummies)

In [19]:
preds = model_gaussian.predict(X_tst[:, 1:])

In [20]:
3.28522171e+14

328522171000000.0

In [21]:
preds

array([[-1.01832404e+14, -2.16734842e+13, -4.93636286e+13,
        -9.39388084e+13, -1.10418172e+14, -4.41424783e+12,
        -2.33424717e+11, -4.32920120e+13, -8.94830505e+13,
        -1.13556892e+14, -8.72977975e+13, -5.07993465e+13,
        -1.06365803e+14, -4.58744057e+13, -1.51261503e+14,
        -2.64994674e+13, -1.18758893e+14, -1.90704464e+14,
        -2.66819856e+14, -4.89544792e+12, -9.88941352e+13,
        -1.16075714e+14, -1.52438730e+11, -1.68199591e+14,
        -1.34937923e+14, -1.35364976e+14, -1.20706406e+14,
        -1.02032801e+14, -1.26449120e+14, -1.22691799e+13,
        -1.31172818e+14, -1.67149993e+14, -1.70460184e+14,
        -1.19828418e+14, -1.89844983e+14, -1.68295827e+14,
        -5.61285640e+13, -1.80380209e+14, -2.74541165e+13,
        -8.95097753e+13, -1.43851253e+14, -3.65994058e+13,
        -1.43485182e+14, -1.32816040e+14, -1.54373544e+14,
        -1.15565834e+14, -2.21584216e+14, -1.53364324e+14,
        -1.11118230e+14, -7.87509970e+13, -3.38714420e+1

In [22]:
acuracy(y_tst, preds), f1_score(y_tst, preds), precision(y_tst, preds)

AttributeError: 'bool' object has no attribute 'sum'

In [ ]:
preds.shape

In [ ]:
preds.argmax(axis=0)

## Naive Bayes

In [23]:
naive_bayes = StatisticalClassifiers(model=NaiveBayesGaussian)

In [24]:
model_naive = naive_bayes.fit(X_trn_, y_trn_dummies)

In [25]:
preds = model_naive.predict(X_tst[:, 1:])

In [26]:
preds

array([[  -363.86850449,  -2771.24518697,   -428.37380054,
          -451.95076544,  -1069.61980443,   -770.68684292,
         -1331.70955696,  -1774.1017649 ,   -180.3858893 ,
         -8418.68091434,  -1596.86243192,  -3050.58744563,
         -5654.56957521,   -443.75278719,  -1307.20805926,
          -784.73721175,  -3818.93047192,  -1931.48104964,
          -580.75884867,  -1497.82538802,   -556.32192925,
          -382.72831663,    -75.50342531,  -4022.6009779 ,
         -7775.24667237,   -445.44848213,  -3141.14900225,
          -909.33516002,  -1341.25877702,   -408.52339381,
         -4446.31080014,  -1687.53758263,   -459.14687004,
         -1698.5949494 ,   -441.66276656,  -1734.22327844,
         -2811.61216576,  -5678.97367924,   -325.4940742 ,
         -4139.26184055,  -2243.13114093,  -1186.94223199,
          -500.10504111,  -1617.21922562,  -1098.32286781,
         -1573.7754618 ,  -3676.97188289,  -5110.16746566,
          -341.25997402,  -7192.62688123,   -392.0578599

In [ ]:
acuracy(y_tst, preds), f1_score(y_tst, preds), precision(y_tst, preds)

In [ ]:
clf = GaussianNB()

In [ ]:
y_trn_dummies[0]

In [ ]:
clf.fit(X_trn_, y_trn_dummies[1])

In [ ]:
preds = clf.predict(X_tst[:, 1:]).reshape([-1, 1])

In [ ]:
preds.shape

In [ ]:
acuracy(y_tst, preds), f1_score(y_tst, preds), precision(y_tst, preds)